In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

In [2]:
# Resetting display option to show all rows
pd.reset_option('display.max_rows')

In [5]:
# Reading the restaurant dataset
restaurant_data = pd.read_csv('/content/Dataset .csv')

In [6]:
# Displaying the first few rows of the dataset
restaurant_data.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [7]:
# Extracting specific columns for analysis
restaurant_data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [10]:
# Displaying the selected columns
df_restaurants = restaurant_data[['Restaurant ID','Restaurant Name','Cuisines','Aggregate rating','Votes']]
df_restaurants

,Restaurant ID,Restaurant Name,Cuisines,Aggregate rating,Votes
0,6317637,Le Petit Souffle,"French, Japanese, Desserts",4.8,314
1,6304287,Izakaya Kikufuji,Japanese,4.5,591
2,6300002,Heat - Edsa Shangri-La,"Seafood, Asian, Filipino, Indian",4.4,270
3,6318506,Ooma,"Japanese, Sushi",4.9,365
4,6314302,Sambo Kojin,"Japanese, Korean",4.8,229
...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,Turkish,4.1,788
9547,5908749,Ceviz A��ac۱,"World Cuisine, Patisserie, Cafe",4.2,1034
9548,5915807,Huqqa,"Italian, World Cuisine",3.7,661
9549,5916112,A���k Kahve,Restaurant Cafe,4.0,901


In [11]:
# Function to describe columns in the dataset
def describe_columns():
    column_info = []
    for col in df_restaurants.columns:
        column_info.append(
            [col,
            df_restaurants[col].dtype,
            df_restaurants[col].isna().sum(),
            round(df_restaurants[col].isna().sum()/len(df_restaurants)*100,2),
            df_restaurants[col].nunique(),
            list(df_restaurants[col].drop_duplicates().sample(2).values)]
        )
    data_description = pd.DataFrame(data=column_info,
                            columns=['Column','Data_Type', 'Missing_Values',
                                        'Percent_Missing', 'Unique_Count', 'Sample_Values'])
    return data_description

In [12]:
# Generating description of columns
describe_columns()

,Column,Data_Type,Missing_Values,Percent_Missing,Unique_Count,Sample_Values
0,Restaurant ID,int64,0,0.00,9551,"[18261309, 18464649]"
1,Restaurant Name,object,0,0.00,7446,"[Superstar Caf��, Dosa Village]"
2,Cuisines,object,9,0.09,1825,"[North Indian, South Indian, Mughlai, Bakery, ..."
3,Aggregate rating,float64,0,0.00,33,"[2.4, 4.0]"
4,Votes,int64,0,0.00,1012,"[908, 219]"


In [13]:
# Dropping rows with missing values
df_restaurants = df_restaurants.dropna()

In [14]:
df_restaurants

,Restaurant ID,Restaurant Name,Cuisines,Aggregate rating,Votes
0,6317637,Le Petit Souffle,"French, Japanese, Desserts",4.8,314
1,6304287,Izakaya Kikufuji,Japanese,4.5,591
2,6300002,Heat - Edsa Shangri-La,"Seafood, Asian, Filipino, Indian",4.4,270
3,6318506,Ooma,"Japanese, Sushi",4.9,365
4,6314302,Sambo Kojin,"Japanese, Korean",4.8,229
...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,Turkish,4.1,788
9547,5908749,Ceviz A��ac۱,"World Cuisine, Patisserie, Cafe",4.2,1034
9548,5915807,Huqqa,"Italian, World Cuisine",3.7,661
9549,5916112,A���k Kahve,Restaurant Cafe,4.0,901


In [15]:
# Renaming columns for clarity
df_restaurants = df_restaurants.rename(columns={'Restaurant_ID': 'restaurant_id'})
df_restaurants = df_restaurants.rename(columns={'Restaurant_Name': 'restaurant_name'})
df_restaurants = df_restaurants.rename(columns={'Cuisines': 'cuisines'})
df_restaurants = df_restaurants.rename(columns={'Aggregate_Rating': 'aggregate_rating'})
df_restaurants = df_restaurants.rename(columns={'Votes': 'votes'})
df_restaurants


,Restaurant ID,Restaurant Name,cuisines,Aggregate rating,votes
0,6317637,Le Petit Souffle,"French, Japanese, Desserts",4.8,314
1,6304287,Izakaya Kikufuji,Japanese,4.5,591
2,6300002,Heat - Edsa Shangri-La,"Seafood, Asian, Filipino, Indian",4.4,270
3,6318506,Ooma,"Japanese, Sushi",4.9,365
4,6314302,Sambo Kojin,"Japanese, Korean",4.8,229
...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,Turkish,4.1,788
9547,5908749,Ceviz A��ac۱,"World Cuisine, Patisserie, Cafe",4.2,1034
9548,5915807,Huqqa,"Italian, World Cuisine",3.7,661
9549,5916112,A���k Kahve,Restaurant Cafe,4.0,901


In [16]:
df_restaurants.duplicated().sum()

0

In [49]:
df_restaurants['Restaurant Name'].duplicated().sum()

1735

In [22]:
df_restaurants['Restaurant Name'].value_counts()

Restaurant Name
Cafe Coffee Day             83
Domino's Pizza              79
Subway                      63
Green Chick Chop            51
McDonald's                  48
                            ..
The Town House Cafe          1
The G.T. Road                1
The Darzi Bar & Kitchen      1
Smoke On Water               1
Walter's Coffee Roastery     1
Name: count, Length: 7437, dtype: int64

In [24]:
df_restaurants = df_restaurants.sort_values(by=['Restaurant Name','Aggregate rating'],ascending=False)
df_restaurants[df_restaurants['Restaurant Name']=="Domino's Pizza"].head()

,Restaurant ID,Restaurant Name,cuisines,Aggregate rating,votes
3031,143,Domino's Pizza,"Pizza, Fast Food",3.7,336
1844,5065,Domino's Pizza,"Pizza, Fast Food",3.6,146
2448,15078,Domino's Pizza,"Pizza, Fast Food",3.6,86
7618,18263236,Domino's Pizza,"Pizza, Fast Food",3.6,24
8437,384,Domino's Pizza,"Pizza, Fast Food",3.6,547


In [26]:
# Removing duplicate rows based on restaurant name
df_restaurants = df_restaurants.drop_duplicates('Restaurant Name',keep='first')
df_restaurants

,Restaurant ID,Restaurant Name,cuisines,Aggregate rating,votes
9523,6000871,�ukura��a Sofras۱,"Kebab, Izgara",4.4,296
3120,18222559,{Niche} - Cafe & Bar,"North Indian, Chinese, Italian, Continental",4.1,492
9334,7100938,wagamama,"Japanese, Asian",3.7,131
9454,6401789,tashas,"Cafe, Mediterranean",4.1,374
4659,18361747,t Lounge by Dilmah,"Cafe, Tea, Desserts",3.6,34
...,...,...,...,...,...
8692,18317511,#Urban Caf��,"North Indian, Chinese, Italian",3.3,49
6998,18336489,#OFF Campus,"Cafe, Continental, Italian, Fast Food",3.7,216
2613,18311951,#InstaFreeze,Ice Cream,0.0,2
9148,18378803,#Dilliwaala6,North Indian,3.7,124


In [27]:
df_restaurants['Restaurant Name'].value_counts()

Restaurant Name
�ukura��a Sofras۱           1
French Toast                1
Fourteen Eleven Tea Cafe    1
Fozzie's Pizzaiolo          1
Frasers                     1
                           ..
Pizza Street                1
Pizza Treat                 1
Pizza Yum                   1
Pizza �� Bessa              1
#45                         1
Name: count, Length: 7437, dtype: int64

In [28]:
# Filtering restaurants with aggregate rating of 4 or above
df_restaurants = df_restaurants[df_restaurants['Aggregate rating']>=4.0]
df_restaurants

,Restaurant ID,Restaurant Name,cuisines,Aggregate rating,votes
9523,6000871,�ukura��a Sofras۱,"Kebab, Izgara",4.4,296
3120,18222559,{Niche} - Cafe & Bar,"North Indian, Chinese, Italian, Continental",4.1,492
9454,6401789,tashas,"Cafe, Mediterranean",4.1,374
9385,6113857,sketch Gallery,"British, Contemporary",4.5,148
1837,18418247,feel ALIVE,"North Indian, American, Asian, Biryani",4.7,69
...,...,...,...,...,...
1468,18408054,19 Flavours Biryani,"Mughlai, Hyderabadi",4.1,84
2484,18233317,145 Kala Ghoda,"Fast Food, Beverages, Desserts",4.2,1606
2292,2100784,11th Avenue Cafe Bistro,"Cafe, American, Italian, Continental",4.1,377
751,2600031,10 Downing Street,"North Indian, Chinese",4.0,257


In [29]:
# Splitting cuisines into individual items
df_restaurants['cuisines'] = df_restaurants['cuisines'].str.split(', ')
df_restaurants

<ipython-input-29-c8eca12bdd27>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_restaurants['cuisines'] = df_restaurants['cuisines'].str.split(', ')


,Restaurant ID,Restaurant Name,cuisines,Aggregate rating,votes
9523,6000871,�ukura��a Sofras۱,"[Kebab, Izgara]",4.4,296
3120,18222559,{Niche} - Cafe & Bar,"[North Indian, Chinese, Italian, Continental]",4.1,492
9454,6401789,tashas,"[Cafe, Mediterranean]",4.1,374
9385,6113857,sketch Gallery,"[British, Contemporary]",4.5,148
1837,18418247,feel ALIVE,"[North Indian, American, Asian, Biryani]",4.7,69
...,...,...,...,...,...
1468,18408054,19 Flavours Biryani,"[Mughlai, Hyderabadi]",4.1,84
2484,18233317,145 Kala Ghoda,"[Fast Food, Beverages, Desserts]",4.2,1606
2292,2100784,11th Avenue Cafe Bistro,"[Cafe, American, Italian, Continental]",4.1,377
751,2600031,10 Downing Street,"[North Indian, Chinese]",4.0,257


In [30]:
# Exploding 'cuisines' column to create multiple rows for each cuisine
df_restaurants = df_restaurants.explode('cuisines',ignore_index=True)
df_restaurants

,Restaurant ID,Restaurant Name,cuisines,Aggregate rating,votes
0,6000871,�ukura��a Sofras۱,Kebab,4.4,296
1,6000871,�ukura��a Sofras۱,Izgara,4.4,296
2,18222559,{Niche} - Cafe & Bar,North Indian,4.1,492
3,18222559,{Niche} - Cafe & Bar,Chinese,4.1,492
4,18222559,{Niche} - Cafe & Bar,Italian,4.1,492
...,...,...,...,...,...
2966,2100784,11th Avenue Cafe Bistro,Italian,4.1,377
2967,2100784,11th Avenue Cafe Bistro,Continental,4.1,377
2968,2600031,10 Downing Street,North Indian,4.0,257
2969,2600031,10 Downing Street,Chinese,4.0,257


In [31]:
# Checking cuisines
df_restaurants['cuisines'].value_counts()

cuisines
North Indian    270
Italian         237
Chinese         200
Continental     199
Cafe            177
               ... 
Pub Food          1
Durban            1
Irish             1
Persian           1
Sunda             1
Name: count, Length: 128, dtype: int64

In [33]:
# # Generating cross tabulation of restaurant names and cuisines

cross_tab_resto_cuisines = pd.crosstab(df_restaurants['Restaurant Name'],
                                df_restaurants['cuisines'])
cross_tab_resto_cuisines

cuisines,Afghani,African,American,Andhra,Arabian,Argentine,Asian,Asian Fusion,Australian,Awadhi,...,Teriyaki,Tex-Mex,Thai,Tibetan,Turkish,Turkish Pizza,Vegetarian,Vietnamese,Western,World Cuisine
Restaurant Name,,,,,,,,,,,,,,,,,,,,,
'Ohana,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10 Downing Street,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11th Avenue Cafe Bistro,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
145 Kala Ghoda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19 Flavours Biryani,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
feel ALIVE,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sketch Gallery,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tashas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# Checking on restaurant name value
cross_tab_resto_cuisines.loc['feel ALIVE'].values

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [35]:
# Sample of Restaurant Names
df_restaurants['Restaurant Name'].sample(20, random_state=101)

1308                    Mrs. Wilkes' Dining Room
2784                                    Baltazar
888                                    Rose Cafe
2713                         Big City Bread Cafe
1162                                Olive Bistro
221                            Transmetropolitan
1403                          Maxims Pastry Shop
1381                                      Meraki
1363                            Mimi's Bakehouse
2466                            Cappuccino Blast
1169                               Oh So Stoned!
1671                       Karak�_y G�_ll�_o��lu
147                                    Via Delhi
209                  Tu-Do Vietnamese Restaurant
258     Tian - Asian Cuisine Studio - ITC Maurya
2649                           Boise Fry Company
247                           Ting's Red Lantern
1170                                Odeon Social
319                                   The Sizzle
690                              Sree Annapoorna
Name: Restaurant Nam

In [36]:
# Measuring similarity between two restaurants using Jaccard similarity score
print(jaccard_score(cross_tab_resto_cuisines.loc["Olive Bistro"].values,
                    cross_tab_resto_cuisines.loc["Rose Cafe"].values))

0.3333333333333333


In [37]:
# Creating a DataFrame to store Jaccard similarity scores between all restaurants
jaccard_distance = pdist(cross_tab_resto_cuisines.values, metric='jaccard')
jaccard_matrix = squareform(jaccard_distance)
jaccard_similarity = 1 - jaccard_matrix
df_jaccard = pd.DataFrame(
    jaccard_similarity,
    index=cross_tab_resto_cuisines.index,
    columns=cross_tab_resto_cuisines.index)

In [38]:
df_jaccard

Restaurant Name,'Ohana,10 Downing Street,11th Avenue Cafe Bistro,145 Kala Ghoda,19 Flavours Biryani,1918 Bistro & Grill,2 Dog,22nd Parallel,3 Wise Monkeys,38 Barracks,...,Zoeys Pizzeria,Zolocrust - Hotel Clarks Amer,Zombie Burger + Drink Lab,Zuka Choco-la,Zunzi's,feel ALIVE,sketch Gallery,tashas,{Niche} - Cafe & Bar,�ukura��a Sofras۱
Restaurant Name,,,,,,,,,,,,,,,,,,,,,
'Ohana,1.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0
10 Downing Street,0.0,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.200000,...,0.0,0.0,0.0,0.000000,0.00,0.200000,0.0,0.0,0.500000,0.0
11th Avenue Cafe Bistro,0.0,0.0,1.000000,0.0,0.0,0.0,0.166667,0.0,0.0,0.333333,...,0.0,0.4,0.0,0.000000,0.00,0.142857,0.0,0.2,0.333333,0.0
145 Kala Ghoda,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.2,0.333333,0.00,0.000000,0.0,0.0,0.000000,0.0
19 Flavours Biryani,0.0,0.0,0.000000,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
feel ALIVE,0.0,0.2,0.142857,0.0,0.0,0.0,0.166667,0.0,0.0,0.600000,...,0.0,0.0,0.0,0.000000,0.00,1.000000,0.0,0.0,0.142857,0.0
sketch Gallery,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.00,0.000000,1.0,0.0,0.000000,0.0
tashas,0.0,0.0,0.200000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.25,0.000000,0.0,1.0,0.000000,0.0


In [39]:
# Sample of Restaurant Names
df_restaurants['Restaurant Name'].sample(20)

557                                  Thai Pepper
292                          The Town House Cafe
1320    Mr Cooper's House & Garden - The Midland
414      The French by Simon Rogan - The Midland
921                                       Replay
1283              NYC - Radisson Blu Plaza Delhi
413                           The Fusion Kitchen
1173                                Odeon Social
1951                          Gemelli Cucina Bar
763                                Sindhi Corner
1304                                      Mughli
2086                             Espress-o-Ville
2896                            Agent Jack's Bar
791                                     Shikhara
2791                                Bakerz Lodge
1447                             Mamu's Infusion
1319    Mr Cooper's House & Garden - The Midland
2923                                      @Mango
553                            That Madras Place
2113                                El Herradero
Name: Restaurant Nam

**# Recommendation**

In [51]:
# Input Initial Restaurant Name for recommendation
initial_restaurant = 'Ooma'

# Calculating similarity scores between the initial restaurant and other restaurants
similarity_scores = df_jaccard.loc[initial_restaurant].sort_values(ascending=False)

# Creating DataFrame to store top 5 similar restaurants
similarity_scores = pd.DataFrame({'Restaurant Name': similarity_scores.index, 'similarity_score': similarity_scores.values})
similarity_scores = similarity_scores[(similarity_scores['Restaurant Name'] != initial_restaurant) & (similarity_scores['similarity_score'] >= 0.7)].head(5)

# Merging with aggregate ratings to get final recommendation
recommended_restaurants = pd.merge(similarity_scores, df_restaurants[['Restaurant Name','Aggregate rating']],how='inner',on='Restaurant Name')
final_recommendations = recommended_restaurants.sort_values('Aggregate rating',ascending=False).drop_duplicates('Restaurant Name',keep='first')
final_recommendations


,Restaurant Name,similarity_score,Aggregate rating
0,Sushi Masa,1.0,4.9
2,Nobu,1.0,4.4
4,Ichiban,1.0,4.3
8,Osaka,1.0,4.2
6,Guppy,1.0,4.1


### **The data above shows the top 5 recommended restaurants with the best ratings. The ratings are curated to include only those that are 4 and above, ensuring the recommendation system provides objectively good ratings.**